In [1]:
from client import start_client
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure

client,w=start_client(5)
client

<Client: 'tcp://127.0.0.1:44358' processes=5 threads=5, memory=33.69 GB>


Client Scheduler: tcp://127.0.0.1:44358 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 33.69 GB


### Main function
- make_classification parameters: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html?highlight=make_classification#sklearn.datasets.make_classification


In [2]:
from main_func import main

batches=100 #number of minibatches
batch_size=1 #size of each minibatch
e=0.02 #tolerance for φ(χ)

#compute samples size
training_s=batches*batch_size
n_samples=int(3*training_s/2) # 1/3 of the samples will be used for testing

#parameters for make classification
dataset_params={
    "n_samples":n_samples,
    "n_features":4,
    "n_informative":2,
    "n_redundant":0,
    "n_repeated":0,
    "n_classes":2,
    "n_clusters_per_class":1,
    "flip_y":0,
    "weights":[0.5,0.5],
    "random_state":17
}
new="yes"

In [ ]:
E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,batches,e)

A new dataset has been created...
coo ([array([ 0.10241722, 10.19187566, 19.7717096 , -8.92807595]), 0.03984063745019917], 1, [4])
Accuracy: 76.000000
coo ([array([ -5.86814294,  -4.93061784,  21.0104263 , -11.16684271]), 9.980198490332903], 1, [2])
Accuracy: 64.000000
coo ([array([  3.86109279, -20.45468102,  28.55820801,  -1.57374929]), 0.05956356969798193], 1, [2])
Accuracy: 72.000000
coo ([array([  7.01838206, -12.73345998,  20.04962837,   2.74826856]), -9.782956115341387], 1, [8])
Accuracy: 72.000000
coo ([array([17.86025415, -1.05123976, 26.67177251, -5.46416601]), 0.04988557590738374], 1, [1])
Accuracy: 82.000000
coo ([array([ -2.9586906 ,  -8.20759438,  31.3894422 , -10.25173868]), 9.86342826481808], 1, [2])
Accuracy: 76.000000
coo ([array([-6.63227359, -5.16454747, 36.474265  , -3.93649583]), 0.05950669619062943], 1, [1])
Accuracy: 80.000000
coo ([array([-4.26763229, -8.79319392, 12.9463704 , 20.45512196]), 9.815604257166239], 1, [5])
Accuracy: 84.000000
coo ([array([-5.480946

### Dataset with no noise y=0
***
#### Low Dimentions
- Balanced
- Unbalanced

In [ ]:
#Balanced weights=[0.5,0,5]
# 3 different e
new="yes"
dataset_params["weights"]=[0.5,0.5]
Bal_acc_e=[]
Bal_rounds_e=[]
for i in [0.01,0.05,0.1]:
    e=i
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,100,e)
    new="no"
    Bal_acc_e.append(Acc)
    Bal_rounds_e.append(Rounds)
# 3 different bach_sizes
Bal_acc_b=[]
Bal_rounds_b=[]
for i in [10,15,100]:
    e=0.01
    batches=i
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,batches,e)
    Bal_acc_b.append(Acc)
    Bal_rounds_b.append(Rounds)
    

In [ ]:
#Unbalanced weights=[0.9,0.1]
# 3 different e
new="yes"
dataset_params["weights"]=[0.9,0.1]
Bal_acc=[]
Bal_rounds=[[] for i in range(3)]
for i in [0.01,0.05,0.1]:
    e=i
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,batches,e)
    new="no"
    UnBal_acc_e.append(Acc)
    UnBal_rounds_e.append(Rounds)
# 3 different bach_sizes
for i in [10,15,100]:
    e=0.01
    batches=i
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,batches,e)
    UnBal_acc_b.append(Acc)
    UnBal_rounds_b.append(Rounds)

In [ ]:
from jupyterthemes import jtplot
from matplotlib.pyplot import figure
jtplot.style(theme='grade3') #light plot theme
#jtplot.style(theme='monokai')
#plots
#1.1.a balanced acc-rounds /e (multiple lines one for each e)
# multiple line plot
figure(figsize=(15,15)) 
plt.subplot(221)
plt.plot(Bal_rounds_e[0], Bal_acc_e[0],label="e=0.01")
plt.plot(Bal_rounds_e[1], Bal_acc_e[1],label="e=0.05")
plt.plot(Bal_rounds_e[2], Bal_acc_e[2],label="e=0.1")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.1.a The behaviour changed with different e \n in a no noise balanced dataset')
plt.legend();

plt.subplot(222)
plt.plot(Bal_rounds_b[0], Bal_acc_b[0],label="baches=10")
plt.plot(Bal_rounds_b[1], Bal_acc_b[1],label="baches=100")
plt.plot(Bal_rounds_b[2], Bal_acc_b[2],label="baches=1000")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.1.b The behaviour changed with the number of minibatches \n in a no noise balanced dataset')
plt.legend();

plt.subplot(223)
plt.plot(UnBal_rounds_e[0], UnBal_acc_e[0],label="e=0.01")
plt.plot(UnBal_rounds_e[1], UnBal_acc_e[1],label="e=0.05")
plt.plot(UnBal_rounds_e[2], UnBal_acc_e[2],label="e=0.1")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.2.a The behaviour changed with different e \n in a no noise Unbalanced dataset')
plt.legend();

plt.subplot(224)
plt.plot(UnBal_rounds_b[0], UnBal_acc_b[0],label="baches=10")
plt.plot(UnBal_rounds_b[1], UnBal_acc_b[1],label="baches=100")
plt.plot(UnBal_rounds_b[2], UnBal_acc_b[2],label="baches=1000")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.2.b The behaviour changed with the number of minibatches \n in a no noise Unbalanced dataset')

plt.show();

#1.2.a unbalanced acc-rounds (multiple lines one for each e)
#1.2.b unbalanced acc-rounds /baches (multiple lines)



### Dataset with low level noise y=
***
#### Low Dimentions
- Balanced
- Unbalanced

In [ ]:
#Balanced weights=[0.5,0,5]
# 3 different e
# 3 different bach_sizes

In [ ]:
#Unbalanced weights=[0.9,0.1]
# 3 different e
# 3 different bach_sizes

In [ ]:
#plots
#2.1.a balanced acc-rounds /e (multiple lines one for each e)
#2.1.b balanced acc-rounds /baches (multiple lines)
#2.2.a unbalanced acc-rounds (multiple lines one for each e)
#2.2.b unbalanced acc-rounds /baches (multiple lines)



### Dataset with average noise level y=
***
#### Low Dimentions
- Balanced
- Unbalanced

In [ ]:
#Balanced weights=[0.5,0,5]
# 3 different e
# 3 different bach_sizes

In [ ]:
#Unbalanced weights=[0.9,0.1]
# 3 different e
# 3 different bach_sizes

In [ ]:
#plots
#3.1.a balanced acc-rounds /e (multiple lines one for each e)
#3.1.b balanced acc-rounds /baches (multiple lines)
#3.2.a unbalanced acc-rounds (multiple lines one for each e)
#3.2.b unbalanced acc-rounds /baches (multiple lines)



### Real Dataset (Mnist)
***

In [ ]:
#load mnist

In [ ]:
# 3 different e
# 3 different bach_sizes

In [ ]:
#plots
#4.1 acc-rounds /e (multiple lines one for each e)
#4.2. acc-rounds /baches (multiple lines)


In [ ]:
from main_func import real_partial
E2,Acc2=real_partial(100)
plt.plot([i for i in range(len(Acc2))],Acc2)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(Rounds,Acc)
Rounds
# plt.legend([i for i in range(len(Acc))], loc='upper left')
